### Week 3: Exploring Linguistic Isolation in LA County
This notebook explores linguistic isolation at the census tract level across LA county using census data. (Author: Aron Walker, with significant attribution to Yoh Kawano's instructions & models)

#### Step 0: Setting up libraries and data

In [ ]:
# Here are the libraries I will use:
import pandas as xp
import geopandas as xg

In [ ]:
# Here is the data I will use:
csv_name = ''
data = xp.read_csv(csv_name)

#### Step 1: Exploring the structure of the census data

In this section, I:

* count the number of rows and columns
* identify the types of data in each column
* look at a few rows to get a sense of the inputs

In [ ]:
# What are its dimensions?
data.shape

In [ ]:
# What types of data are present?
data.info(verbose=True, show_counts=True)

In [ ]:
# What do the first few rows look like?
data.head()

In [ ]:
# What do the last few rows look like?
data.tail()

In [ ]:
# What do some random rows look like?
data.sample()

#### Step 2: Limiting the data to what interests me

For each census tract, I really only want:

* unique identifier for the tract
* total population
* linguistic isolation score

Also, I want the columns to have legible names.

In [ ]:
# Here are the columns I want in census-speak:
columns_of_interest = ['Geo_FIPS',
                   'SE_T004_019',
                   'SE_T004_001',
                   'SE_T004_003',
                   'SE_T004_005',
                   'SE_T004_007',
                   'SE_T004_009',
                   'SE_T004_011',
                   'SE_T004_013',
                   'SE_T004_015',
                   'SE_T004_017']

# Here is the data with just those columns:
data2 = data[columns_of_interest]

# And let's name those columns with plain language
data2.columns_of_interest = ['FIPS',
'TotalPop',
'Non Hispanic',
'Non Hispanic White Alone',
'Non Hispanic Black Alone',
'Non Hispanic American Indian and Alaska Native Alone',
'Non Hispanic Asian Alone',
'Non Hispanic Native Hawaiian and Pacific Islander Alone',
'Non Hispanic Other Alone',
'Non Hispanic Multi Race',
'Hispanic']

Finally, I only want census tracts with people.

In [ ]:
# Drop the rows (tracts) without people
data2 = data2[data2['TotalPop']>0]

#### Step 3: Explore the data's distribution